# Neighborhood Council Population Density Calculation

This is an initial exploration to build a population density measure for the NC's

Steps in this notebook:

1.  Setup
2.  Create geodataframe/dataframe from cleaned data and [census](https://data.lacity.org/Community-Economic-Development/Census-Data-by-Neighborhood-Council/nwj3-ufba)
3.  Examine the data
4.  Compute the measure
5.  Show measure as choropleth
6.  So what (next steps)

# 1 - Setup

In [1]:
%run start.py

import nc

# 2 - Get Data Files

Using two data files:

  - The Certified Neighborhoods cleaned up in NC-service-regions.ipynb
  - [Census](https://data.lacity.org/Community-Economic-Development/Census-Data-by-Neighborhood-Council/nwj3-ufba) data from data.lacity.org
  
**Note** - This is 2010 Census data.  Probably need to look for 2020.

In [2]:
#neighborhoods_gdf = gpd.read_file('../data/neighborhoods/geo_export_6da7921e-60c3-4768-ba2c-486e681d16fe.shp')
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')

neighborhoods_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

In [3]:
neighborhood_census_df = pd.read_csv('../data/neighborhoods/Census_Data_by_Neighborhood_Council.csv')

# 3 - Quick Looksie

In [4]:
info_gdf = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [5]:
info_df = Output(layout={'border': '1px solid black',
                            'width': '50%'})

In [6]:
with info_gdf:
    display(HTML('<center><b>gdf info()</b></center>'))
    display(neighborhoods_gdf.info())

In [7]:
with info_df:
    display(HTML('<center><b>df info()</b></center>'))
    display(neighborhood_census_df.info())

In [8]:
HBox([info_gdf, info_df])

Well, there's a discrepancy here.  The census data has 97 NC's and the certified dataset has 99 (I think the right number is 99).

Not going to agonize over this at this stage but want to understand things.  Adjusting for what matches as this stage should be good enough for now.

In [9]:
names_set = set(neighborhoods_gdf.name)

In [10]:
NC_Names_set = set(neighborhood_census_df.NC_Name)

In [11]:
names_set.difference(NC_Names_set)

{'ECHO PARK NC', 'HISTORIC CULTURAL NORTH NC', 'NOHO NC', 'NORTH WESTWOOD NC'}

In [12]:
in_both = names_set.intersection(NC_Names_set)
in_both;  #delete the ; if you want to see names in both

So this will build a new gdf where neighborhoods are in both.  neighborhoods_gdf is still the baseline.

In [13]:
neighborhoods_in_both = neighborhoods_gdf.query(f"name in @in_both")

In [14]:
type(neighborhoods_in_both)

geopandas.geodataframe.GeoDataFrame

In [15]:
neighborhoods_in_both.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 95 entries, 0 to 98
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   OBJECTID        95 non-null     int64   
 1   name            95 non-null     object  
 2   WADDRESS        95 non-null     object  
 3   DWEBSITE        95 non-null     object  
 4   DEMAIL          94 non-null     object  
 5   DPHONE          95 non-null     object  
 6   nc_id           95 non-null     int64   
 7   CERTIFIED       95 non-null     object  
 8   TOOLTIP         95 non-null     object  
 9   NLA_URL         95 non-null     object  
 10  service_region  95 non-null     object  
 11  region_id       95 non-null     object  
 12  color_code      95 non-null     object  
 13  geometry        95 non-null     geometry
dtypes: geometry(1), int64(2), object(11)
memory usage: 11.1+ KB


In [16]:
neighborhood_census_df.rename(columns={'NC_Name': 'name'}, inplace=True)

Use standard pd.merge to create the gdf for the choropleth.

In [17]:
neighborhood_merged = pd.merge(neighborhoods_in_both, neighborhood_census_df, how="left", on=["name"])

In [18]:
type(neighborhood_merged)

geopandas.geodataframe.GeoDataFrame

# 4 - Compute the Measure

Computation is simple.  Use the geometry of the NC to compute area in miles squared.

For the density I'm simply using total population.  I suspect it would be interesting to examine some of the other ethnic measures?  Maybe a nice pull down to select?  Ah... for another day.

In [19]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

def square_miles(geo):
    square_meters = abs(geod.geometry_area_perimeter(geo)[0])
    return (square_meters * 10.764) / 27878000

In [20]:
neighborhood_merged['sq_miles'] = neighborhood_merged.apply(lambda row: square_miles(row.geometry), axis=1)

In [21]:
neighborhood_merged['density'] = neighborhood_merged.apply(lambda row: row['Total Population'] / row['sq_miles'], axis=1)

Remember I like to look at one of the values.

In [22]:
neighborhood_merged.iloc[27]

OBJECTID                                                           29
name                                          GREATER CYPRESS PARK NC
WADDRESS                               https://www.cypressparknc.com/
DWEBSITE                                 https://empowerla.org/gcpnc/
DEMAIL                                            GCPNC@EmpowerLA.org
DPHONE                                                   213-978-1551
nc_id                                                             102
CERTIFIED                                                  2002-11-19
TOOLTIP                                       GREATER CYPRESS PARK NC
NLA_URL                      navigatela/reports/nc_reports.cfm?id=102
service_region                               REGION 8 - NORTH EAST LA
region_id                                                           8
color_code                                                    #FF8C00
geometry            POLYGON ((-118.2118080003933 34.0892640001135,...
Total Population    

Some sanity checking on the data before we generate the display.

In the real world we'll have to do some more work on this data!

In [23]:
neighborhood_merged.density.max()

47469.29021232103

In [24]:
neighborhood_merged.density.min()

1174.4250551603595

In [25]:
len(neighborhood_merged)

95

# 5 - Display the Choropleth

In [26]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(34.05, -118.25), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

#map_display.add_control(LayersControl())
#map_display += nc_layer
map_display

Map(center=[34.05, -118.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

refer to : https://www.youtube.com/watch?v=wjzAy_yLrdA

In [27]:
from ipyleaflet import Choropleth, Map
from branca.colormap import linear
a_geojson = json.loads(neighborhood_merged.to_json())

population_density = dict(zip(neighborhood_merged['name'].tolist(), neighborhood_merged['density'].tolist()))
for i in a_geojson['features']:
    i['id'] = i['properties']['name']

layer = Choropleth(
                    geo_data=a_geojson,
                    choro_data=population_density,
                    colormap=linear.YlOrRd_09, #linear.Blues_05,
                    style={'fillOpacity': 1.0, "color":"black"},)
                    #key_on="name")

map_display.add_layer(layer)

I need to revisit a tooltip type popup.  For now this will work.

In [28]:
geo_json = GeoJSON(
    data=a_geojson,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    name='NCs'
)

html = HTML('''Hover over a district''')
html.layout.margin = '0px 20px 20px 20 px'
control = WidgetControl(widget=html, position='bottomright')

def update_html(feature, **kwargs):
    html.value = '''<h3><b>NC: {}</b></h3>
                    <h4>Area (sq miles): {}</h4>
                    <h4>Total Pop: {}</h4>
                    <h4>Density: {}</h4>'''.format(feature['properties']['name'],
                                                           feature['properties']['sq_miles'],
                                                           feature['properties']['Total Population'],
                                                           feature['properties']['density'])
    
map_display.add_control(control)  # does += work for this?

layer.on_hover(update_html)

# 6 - So What?

I say this tounge in cheeck.  Things to think about:

  1. Should we examine measures besides total population?
  2. Does it make sense to extend the 311 data as we did with the service regions?
  3. Do we just use this to select an NC then query 311 (or ...)?
  
